### This solution notebook is not yet finished and things do not work yet as they should. 

### Task 1

Since there is only one set of constraints it is not difficult to choose which constraints to relax.
$$
\begin{array}[ll]
\text{min} &\sum_{j=1}^n[c_j - \sum_{i=1}^n \lambda_i a_{ij}]x_j + \sum^n_{j=1}\lambda_i\\
\text{s.t.}&x_j\in \{0,1\} \qquad j=1,...,n 
\end{array}
$$

Defining $C_j = [c_j - \sum_{i=1}^m \lambda_i a_{ij}]$ $j=1,...,n$
i.e. $C_j$ is the coefficient of $x_j$ in the objective function of LR we have that LR becomes:
$$
\begin{array}[ll]
\text{min} &\sum_j(C_jx_j + \lambda_i)\\
\text{s.t.}& x_j\in \{0,1\} \qquad j=1,...,n 
\end{array}
$$

Now the solution ($x_j$) to LR can be found by inspection, namely: 
$$
x_j=
\begin{cases}
 1 &if C_j \leq 0\\
  0 &otherwise
\end{cases}
$$
with the solution value ($z_{LB}$) of LR being given by:
$$z_{LB} = C_jx_j + \lambda_i$$
where $z_{LB}$ is a lower bound on the optimal solution to the original SCP. 

In [123]:
import numpy as np
from pyscipopt import Model, quicksum



c = np.array([ 2,3,4,5 ])
b = np.array([1,1,1])
A = np.array([[ 1, 0, 1, 0 ],
            [1, 0, 0, 1],
            [0, 1, 1, 1]])

def solve_scip():
    # Model
    model = Model("lagrangian relaxation")
    # Create decision variables
    x = {}
    for i in range(4):
        x[i] = model.addVar(name="x"+str(i), vtype="C", lb=0.0, ub=None, pricedVar = False)
    # The objective is to maximize (the default)
    # Unecessary if we had written the obj coefficient for all vars above
    model.setObjective(quicksum(c[i]*x[i] for i in range(4)), "minimize")
    # Add constraints to the model
    model.addCons(x[0] + x[2] >= 1, "c1")
    model.addCons(x[0] + x[3] >= 1, "c2")
    model.addCons(x[1] + x[2]+x[3] >= 1, "c3")
    # Solve
    model.optimize()
    # Let’s print the solution
    if model.getStatus() == "optimal":
        z_LP = model.getObjVal()
        x_LP = np.array([model.getVal(v) for v in model.getVars()])
        return x_LP,z_LP
    else:
        print("Problem could not be solved to optimality")
        

In [124]:
solve_scip()

(array([1., 1., 0., 0.]), 5.0)

In [118]:

def solve_lagrangian_scip(lambdas):
    C = c - lambdas * A[2,:]
    # Model
    model = Model("lagrangian relaxation")
    # Create decision variables
    x = {}
    for i in range(4):
        x[i] = model.addVar(name="x"+str(i), vtype="C", lb=0.0, ub=None, pricedVar = False)
    # The objective is to maximize (the default)
    # Unecessary if we had written the obj coefficient for all vars above
    model.setObjective(quicksum(C[i]*x[i] for i in range(4))+lambdas, "minimize")
    # Add constraints to the model
    model.addCons(x[0] + x[2] >= 1, "c1")
    model.addCons(x[0] + x[3] >= 1, "c2")
    # Solve
    model.optimize()
    # Let’s print the solution
    if model.getStatus() == "optimal":
        z_LLBP = model.getObjVal()
        x_LLBP = np.array([model.getVal(v) for v in model.getVars()])
        return x_LLBP,z_LLBP
    else:
        print("Problem could not be solved to optimality")
        
    

### Task 2

A common fallacy in Lagrangean relaxation is to believe that, if the solution to LR
is feasible for the original problem, then it is also optimal for the original problem.
This is incorrect. 

In [125]:
lambdas = 1
solve_lagrangian_scip(lambdas)

[2 2 3 4]


(array([1., 0., 0., 0.]), 3.0)

In [126]:
lambdas = 2
solve_lagrangian_scip(lambdas)

[2 1 2 3]


(array([1., 0., 0., 0.]), 4.0)

ValueError: shapes (1,) and (4,) not aligned: 1 (dim 0) != 4 (dim 0)

A solution $x$ to a Lagrangean lower bound program is only optimal for the
original problem if:
(a) $x$ is feasible for the original problem; and
(b) $cx = [cX + \lambda(b - AX)]$ i.e. $\lambda(b - Ax) = 0$
If we are relaxing equality
constraints ($Ax=b$) then any solution to the lagrangean lower bound program which is
feasible for the original problem automatically satisfies both (a) and (b) above and so is
optimal. 

### Task 4

In [88]:
def heuristic_sol(x):
    x_h=np.copy(x)
    covered = A @ x_h-b >= 1
    for i in range(len(covered)):
        if not covered[i]:
            for j in range(len(A[i,:])):
                if A[i,j] >0:
                    x_h[j]=1
                    break
    z_UB = c @ x_h
    #print(x_h,z_UB)
    return z_UB

In [143]:
%matplotlib inline
import matplotlib.pyplot as plt

def solve_lagrangian_dual(mu=2, iterations=10):
    zs_LB=[]
    zs_UB=[]
    lambdas_ = []
    lambdas = 0
    for t in range(iterations):
        x, z_LLBP = solve_lagrangian_scip(lambdas)
        z_UB = heuristic_sol(x)
        gamma = b[2] - A[2,:] @ x
        theta = mu * ( z_UB - z_LLBP )/(gamma**2)
        lambdas = max(lambdas + theta * gamma, 0)
        zs_UB.append(z_UB); zs_LB.append(z_LLBP); lambdas_.append(lambdas)
    
    plt.subplot(2, 1, 1)
    plt.plot(range(iterations), zs, 'o-')
    plt.ylabel('z_LLBP')
    plt.xlabel('iteration')

    plt.subplot(2, 1, 2)
    plt.plot(range(iterations), lambdas_, 'o-')
    plt.xlabel('iteration')
    plt.ylabel('lambda')

In [144]:
solve_lagrangian_dual()

[2 3 4 5]
[ 2. -3. -2. -1.]
Problem could not be solved to optimality


TypeError: 'NoneType' object is not iterable

# Task 2

We consider the following problem (Fisher M., [An Applications Oriented
Guide to Lagrangian Relaxation](http://www.cs.uleth.ca/~benkoczi/OR/read/lagrange-relax-introduct-fisher85.pdf) Interfaces, 15:2, 1985):

$$
\begin{array}{lllll}
 z_P=&\text{max} &16x_1+10x_2+4x_4\\
&\text{s.t.}&8x_1+2x_2+x_3+x_4\leq 10\\
&&x_1+x_2\leq 1\\
&&x_3+x_4\leq 1\\
&&0\leq x\leq 1 \qquad \text{and integer}
\end{array}
$$

There are three major questions to design a Lagrangian-relaxation-based system:
a. which constraints should be relaxaed
b. how to compute good multipliers $\lambda$
c. how to deduce a good feasible solution to the original problem, given a solution to the Lagrangian relaxation problem.

The answers are:
a. those whose relaxation makes the problem significantly easy but not too easy
b. subgradient procedure
c. problem specific heuristics


## Subtask 2.1
If we relax the first constraint with multiplier $\lambda\geq 0$ the corresponding Lagrangian relaxation problem becomes:

$$
\begin{array}{lllll}
 z_P=&\text{max} &(16-8\lambda)x_1+(10-2\lambda)x_2+(0-\lambda)x_3+(4-4\lambda)x_4+10\lambda\\
&\text{s.t.}&x_1+x_2\leq 1\\
&&x_3+x_4\leq 1\\
&&0\leq x\leq 1 \qquad \text{and integer}
\end{array}
$$


For a given $\lambda$ we could solve the problem by inspection: 
- between $x_1$ and $x_2$ set to 1 the variable with the largest cost coefficient in the objective function; 
- between $x_1$ and $x_2$ set to 1 the variable with the largest cost coefficient in the objective function.
However let's use the SCIP procedure developed above.

In [3]:
import numpy as ny
from pyscipopt import Model


c = np.array([ 16,10,0,4 ])
b = np.array([10,1,1])
A = np.array([[ 8, 2, 1, 1 ],
            [1, 1, 0, 0],
            [0, 0, 1, 1]])



def solve_scip():
    # Model
    model = Model("lagrangian relaxation")
    # Create decision variables
    
    for i in range(4):
        col = model.Column()
        for j in range(3):
            col.addTerms(A[i,j])
        x[i] = model.addVar(name="x"+str(i), vtype="C", lb=0.0, ub=None, pricedVar = False, column=col)
    
    model.update()
    model.writeProblem("prob.lp")
    # The objective is to maximize (the default)
    # Unecessary if we had written the obj coefficient for all vars above
    model.setObjective(quicksum(c[i]*x[i] for i in range(4)), "maximize")

    # Solve
    model.optimize()
    # Let’s print the solution
    if model.getStatus() == "optimal":
        z_LP = model.getObjVal()
        x_LP = np.array([model.getVal(v) for v in model.getVars()])
        return x_LP,z_LP
    else:
        print("Problem could not be solved to optimality")
        

NameError: name 'np' is not defined